<a href="https://colab.research.google.com/github/gopalm-ai/Quantum_Machine_Learning_An_Applied_Approach/blob/main/qft_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Import Libraries
! pip3 install pyquil
! pip3 install qutip

     |████████████████████████████████| 14.6 MB 6.7 MB/s 


In [2]:
from pyquil import get_qc, Program
from pyquil.gates import CNOT, Z, MEASURE
from pyquil.api import local_forest_runtime

In [3]:
import cirq
print(cirq.__version__)

0.13.1


In [4]:
! [ -z "$(which qvm)" ] &&\
  apt update &&\
  apt install jq &&\
  export LATEST_FOREST_SDK_VERSION=$(curl -s https://downloads.rigetti.com/qcs-sdk/versions | jq -r '.versions[0].sdk') &&\
  curl -f https://downloads.rigetti.com/qcs-sdk/forest-sdk-$LATEST_FOREST_SDK_VERSION-linux-deb.tar.bz2 -o $PWD/forest-sdk-$LATEST_FOREST_SDK_VERSION-linux-deb.tar.bz2 &&\
  tar -xf forest-sdk-$LATEST_FOREST_SDK_VERSION-linux-deb.tar.bz2 &&\
  ./forest-sdk-$LATEST_FOREST_SDK_VERSION-linux-deb/forest-sdk-$LATEST_FOREST_SDK_VERSION-linux-deb.run &&\
  quilc --version &&\
  qvm --version

In [9]:
import subprocess

subprocess.Popen(["qvm", "--quiet", "-S"])
subprocess.Popen(["quilc", "--quiet", "-R"])

In [13]:
from pyquil import Program, get_qc
from pyquil.gates import *
import cmath
import matplotlib.pyplot as plt
import numpy as np
from pyquil.api import QCSClientConfiguration, EngagementManager
from pyquil.api import WavefunctionSimulator
from pyquil.api import QVM
from qutip import Bloch
from tempfile import mkdtemp
qvm = QVM()

In [14]:
prog = Program(Z(0), CNOT(0, 1))
with local_forest_runtime():
  qvm = get_qc('9q-square-qvm')
  #results = qvm.run_and_measure(prog, trials=10)
  results = qvm.run(prog)
  bitstring = results.readout_data.get("ro")
  print(bitstring)

None


/usr/local/lib/python3.7/dist-packages/pyquil/api/_quantum_computer.py:957: RuntimeWarning: Unable to start qvm server, since the specified port 5000 is in use.
  warnings.warn(RuntimeWarning(warning_msg))
/usr/local/lib/python3.7/dist-packages/pyquil/api/_quantum_computer.py:964: RuntimeWarning: Unable to start quilc server, since the specified port 5555 is in use.
  warnings.warn(RuntimeWarning(warning_msg))


In [21]:
import math
from math import pi

In [22]:
# Quantum Fourier Transform
state_prep = Program(X(0))
qft = Program()
qft += H(0)
qft += CPHASE(pi/2, 1, 0)
qft += H(1)
qft += CPHASE(pi/4, 2, 0)
qft += CPHASE(pi/2, 2, 1)
qft += H(2)

In [23]:
# Add dummy qubits
from pyquil.api import WavefunctionSimulator
add_dummy_qubits = Program(I(1), I(2))
# Identity gate I has no effect
wf_sim = WavefunctionSimulator()
wavefunction = wf_sim.wavefunction(state_prep + add_dummy_qubits)
print(wavefunction)

(1+0j)|001>


In [24]:
# 3 qubit QFT Program: DFT of [0, 1, 0, 0, 0, 0, 0, 0], using PyQuil:
def qft1(q0, q1, q2):
  p = Program()
  p += [SWAP(q0, q2), 
        H(q0), 
        CPHASE(-pi/2.0, q0, q1),
        H(q1),
        CPHASE(-pi/4.0, q0, q2),
        CPHASE(-pi/2.0, q1, q2),
        H(q2)]
  return p

print(qft1(0, 1, 2))

SWAP 0 2
H 0
CPHASE(-pi/2) 0 1
H 1
CPHASE(-pi/4) 0 2
CPHASE(-pi/2) 1 2
H 2



In [25]:
compute_qft_prog = state_prep + qft1(0, 1, 2)
wavefunction = wf_sim.wavefunction(compute_qft_prog)
print(wavefunction.amplitudes)

[ 3.53553391e-01+0.j          2.50000000e-01-0.25j
  2.16489014e-17-0.35355339j -2.50000000e-01-0.25j
 -3.53553391e-01+0.j         -2.50000000e-01+0.25j
 -2.16489014e-17+0.35355339j  2.50000000e-01+0.25j      ]


In [27]:
# Verify qft by inverse FFT
import numpy 
from numpy.fft import ifft
ifft(wavefunction.amplitudes, norm="ortho")

array([ 0.00000000e+00+0.00000000e+00j,  1.00000000e+00+5.45603965e-17j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00-1.53080850e-17j,
        0.00000000e+00+0.00000000e+00j, -7.85046229e-17-2.39442265e-17j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00-1.53080850e-17j])